# Benchmarking the Predictive Services

<h1 style="color:red">Start by setting your S3 path!!!</h1>

The predictive service needs a location for saving its configuration and state: nodes will be reading data from this location. This notebook will deploy on S3, so naturally this shared location will be an S3 path. If you were deploying on-premises, you'd be using some shared network location.

Please set the s3_path to a bucket to which you can write using your configured AWS credentials! Below is my path - you won't be able to read/write to it so change it to something on your bucket space.

In [ ]:
s3_path = "s3://gl-guyrap-testing/ps_benchmark" # You should create an empty S3 bucket/path .

<h1 style="color:red">End by uncommenting and running the last cell!!!</h1>

I am assuming that after setting your `s3_path`, you'll just run all the cells. This will create a new predictive service on AWS - but it will not shut it down when you're finished (= you'll still get billed for it). **Go to the last cell, uncomment it and run the `deployment.terminate_service()` command to stop getting billed.**

Of course - it'd be better for you to go through the entire notebook step by step. But I know you! `¯\_(ツ)_/¯`

## Prerequisites and Setup

GraphLab Create has to be installed. This notebook was tested with version 1.9.

Apache Bench is a command line tool for benchmarking web services. It is part of the Apache HTTP Server package.
The package can be [downloaded from Apache's website](http://httpd.apache.org/download.cgi). However, I'll be using Apache Bench for Mac [which is available here](https://github.com/radiospiel/ApacheBench-Lion/raw/master/ab).|

Some configuration variables should be set in order to create the predictive service properly.

### Verify GraphLab is installed

In [ ]:
try:
    import graphlab # same import as in the predictive services documentation
    gl = graphlab # my favourite alias
    print graphlab.version
except:
    raise RuntimeError("GraphLab Create is missing!")

### Verify AWS credentials are set

In [ ]:
gl.aws.get_credentials()
pass

### Verify `ab` is available in the system path

We'll be benchmarking the predictive services using Apache Bench. <a href="#Benchmark-using-Apache-Bench-(ab)">Go to the relevant section below</a> for more details.

In [ ]:
import os
try:
    ab_present = os.system("which ab") == 0
except:
    ab_present = False

if ab_present:
    print "ab is present, benchmarking can continue!"
else:
    print "ab (Apache Bench) is missing! You can continue running the next cells, but benchmarking u"

## Create a Predictive Service on Amazon EC2

From this point on, we'll be using code example from the user guide:
https://dato.com/learn/userguide/deployment/pred-getting-started.html


In [ ]:
# You can change the following two variables.
region = 'us-west-2' # may affect client-side latency - the round-trip time depends on distance from region
instance_type = 'm3.xlarge' # may affect server-side latency and throughput - and also how much you'll pay :)

ec2 = graphlab.deploy.Ec2Config(region=region,
                                instance_type=instance_type)

"""
ec2 = graphlab.deploy.Ec2Config(region=region,
                                instance_type=instance_type,
                                aws_access_key_id='YOUR_ACCESS_KEY',
                                aws_secret_access_key='YOUR_SECRET_KEY')
"""

ec2

In [ ]:
# If you had already deployed a service in the path, you can reconnect to it:
# deployment = gl.deploy.predictive_service.load(s3_path)

In [ ]:
deployment = graphlab.deploy.predictive_service.create(
    name="ps-benchmark",
    ec2_config=ec2,
    state_path = s3_path,
    num_hosts=1,
    description="benchmarking the predictive services"
)

## Create a predictive model

In [ ]:
model_path = "recommender_movie_ratings"
if os.path.exists(model_path):
    model = gl.load_model(model_path)
else:
    from urllib import urlretrieve
    data_url = "https://s3.amazonaws.com/dato-datasets/movie_ratings/sample.small"
    data_filename = data_url.rsplit("/", 1)[-1]
    if not os.path.exists(data_filename):
        urlretrieve(data_url, data_filename)

    data = graphlab.SFrame.read_csv(data_filename,delimiter='\t',column_type_hints={'rating':int})
    model = graphlab.popularity_recommender.create(data, 'user', 'movie', 'rating')
    model.save(model_path)

In [ ]:
deployment.add('recs', model)

In [ ]:
test_recs = deployment.test_query('recs',
                        method='recommend',
                        data={ 'users': [ 'Jacob Smith' ] })
print test_recs

In [ ]:
deployment.apply_changes()

In [ ]:
recs = deployment.query('recs',
                        method='recommend',
                        data={ 'users': [ 'Jacob Smith' ] })
print recs

In [ ]:
api_key = deployment.api_key
load_balancer_dns_name = deployment._environment.load_balancer_dns_name
query_endpoint = 'http://%s/query/recs' % (load_balancer_dns_name)

In [ ]:
curl_cmd = """curl -u api_key:%s \
-d '{ "data": {"method": "recommend", "data": { "users": [ "Jacob Smith" ] } } }' \
%s""" % (api_key, query_endpoint)
print curl_cmd

In [ ]:
! $curl_cmd

## Benchmark using Apache Bench (`ab`)

Apache Bench is a command line tool for benchmarking web services. It is part of the Apache HTTP Server package.

The package can be downloaded from here:
http://httpd.apache.org/download.cgi

However, I'll be using Apache Bench for Mac which I obtained from:
https://github.com/radiospiel/ApacheBench-Lion/raw/master/ab

A nice tutorial for Apache Bench can be found here:
https://www.petefreitag.com/item/689.cfm

The benchmark is simple: make the same query many times, and see how good is the predictive service's throughput and latency.

The request's json is first written to a text file:

In [ ]:
benchmark_request_data_filename = 'benchmark_request_data.txt'
with open(benchmark_request_data_filename, 'wb') as f:
    f.write('''{ "data": {"method": "recommend", "data": { "users": [ "Jacob Smith" ] } } }''')

In [ ]:
! cat 'benchmark_request_data.txt'

In [ ]:
concurrent = 4
nr_requests = 40
if ab_present:
    ab_cmd = "! ab -A api_key:%s -p %s -T application/json -c %d -n %d %s" % (
        api_key,
        benchmark_request_data_filename,
        concurrent,
        nr_requests,
        query_endpoint
    )
else:
    ab_cmd = ""

In [ ]:
! $ab_cmd

## Terminate the Predictive Service (release EC2 resources)

Uncomment the following line and run this cell to terminate the predictive service. This will release (free) the EC2 instances, so your billing would stop.

In [ ]:
# deployment.terminate_service()